In [1]:
# Initial Imports

import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
import time
import os
from pathlib import Path
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
# Import flair Sentence to process input text
from flair.data import Sentence
import regex as re
from sentiment_functions import clean_text, score_flair
import datetime
# BDay is business day, not birthday...
from pandas.tseries.offsets import BDay




2022-01-18 15:23:43,885 loading file C:\Users\sjufa\.flair\models\sentiment-en-mix-distillbert_4.pt
2022-01-18 15:23:53,910 loading file C:\Users\sjufa\.flair\models\sentiment-en-mix-distillbert_4.pt


In [4]:
# Taking user input for number of days to retrieve data for, then creating the date range to iterate through for the API call


def getTwitterData(hashtag):
        
    tweets_df = pd.DataFrame()
    numDays = int(input('How many days would you like to retrieve twitter data for?'))
    #hashtag = input('What hashtag would you like to retrieve data for?')
    today = datetime.datetime.today()
    end = today 
    start = end - pd.Timedelta(days = numDays)
    
    #Since we are retrieving data we want to correlate with markets, we are using 'bdate_range' to only return business days                
    date_range = pd.bdate_range(start=start, end=end, freq="D", tz='UTC') + pd.Timedelta(hours=5)
        
    
    #Iterating through the date_range with an API call for each day to maximize data returned 
    # @TODO -- use function to limit API calls to not exceed limits
    try:
        for i in range(0, len(date_range) -1, 2):
    
            start_date = date_range[i]
            end_date = date_range[i+52] 
    
    
            start_date = pd.Timestamp.isoformat(start_date)
            end_date = pd.Timestamp.isoformat(end_date)
    
    
            # Making the API call to retrieve tweets 

            source = "https://twitter32.p.rapidapi.com/getSearch"

            # These parameters need to be set before running API call... could in the future receive inputs from user
            ## TODO -- if input is received, datetime will need to be formatted correctly for API call

            hashtag = hashtag

            querystring = {"hashtag": hashtag, "start_date": start_date,"end_date": end_date,"lang":"en"}
            headers = {
            'x-rapidapi-host': "twitter32.p.rapidapi.com",
            'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
            }
   
        
            response = requests.get(source, headers=headers, params=querystring).json()
            df = pd.DataFrame(response['data']['tweets']).T
            tweets_df = pd.concat([df, tweets_df])
                
        
            if i >= 20 and i % 20 == 0:
                time.sleep(60)
        
            numDays -= 1
        
    except Exception as e:
        print('Sorry, there was an', e.__class__,f'error in your request.  Please re-run your inquiry from {numDays} days'\
        f'ago for hashtag {hashtag}.  Your successful tweets searched have been saved to the dataframe.')
        
    finally:        
        return tweets_df
        
    return tweets_df

crm_df = getTwitterData('$BBY')

display(crm_df)
        



How many days would you like to retrieve twitter data for? 400


,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,quote_count,conversation_id,conversation_id_str,favorited,retweeted,lang,supplemental_language,possibly_sensitive,possibly_sensitive_editable,card
1478095145824231430,Mon Jan 03 20:05:19 +0000 2022,1478095145824231430,1478095145824231430,"Bull Forecasts\nJanuary 03, 2022 12:00:00 PDT\...",False,"[0, 210]","{'hashtags': [], 'symbols': [{'text': 'ESSC', ...","<a href=""https://help.twitter.com/en/using-twi...",None,None,...,0,1478095145824231430,1478095145824231430,False,False,en,None,NaN,NaN,NaN
1479167228901675008,Thu Jan 06 19:05:23 +0000 2022,1479167228901675008,1479167228901675008,"Bull Forecasts\nJanuary 06, 2022 11:00:00 PDT\...",False,"[0, 200]","{'hashtags': [], 'symbols': [{'text': 'ESSC', ...","<a href=""https://help.twitter.com/en/using-twi...",None,None,...,0,1479167228901675008,1479167228901675008,False,False,en,None,NaN,NaN,NaN
1471587169164673024,Thu Dec 16 21:04:56 +0000 2021,1471587169164673024,1471587169164673024,"Bull Forecasts\nDecember 16, 2021 13:00:00 PDT...",False,"[0, 211]","{'hashtags': [], 'symbols': [{'text': 'SPY', '...","<a href=""https://help.twitter.com/en/using-twi...",None,None,...,0,1471587169164673024,1471587169164673024,False,False,en,None,NaN,NaN,NaN
1471572286050541568,Thu Dec 16 20:05:48 +0000 2021,1471572286050541568,1471572286050541568,"Bull Forecasts\nDecember 16, 2021 12:00:00 PDT...",False,"[0, 210]","{'hashtags': [], 'symbols': [{'text': 'IWM', '...","<a href=""https://help.twitter.com/en/using-twi...",None,None,...,0,1471572286050541568,1471572286050541568,False,False,en,None,NaN,NaN,NaN
1468206216568528910,Tue Dec 07 13:10:14 +0000 2021,1468206216568528910,1468206216568528910,$CRM [15s. delayed]: Issued Press Release on D...,False,"[0, 229]","{'hashtags': [], 'symbols': [{'text': 'CRM', '...","<a href=""https://vymcfued1m.execute-api.us-eas...",None,None,...,0,1468206216568528910,1468206216568528910,False,False,en,None,False,True,NaN
1466002278503047170,Wed Dec 01 11:12:34 +0000 2021,1466002278503047170,1466002278503047170,$CRM [15s. delayed]: Issued Press Release on D...,False,"[0, 177]","{'hashtags': [], 'symbols': [{'text': 'CRM', '...","<a href=""https://vymcfued1m.execute-api.us-eas...",None,None,...,0,1466002278503047170,1466002278503047170,False,False,en,None,False,True,NaN
1445072606193983502,Mon Oct 04 17:05:32 +0000 2021,1445072606193983502,1445072606193983502,$CRM [15s. delayed]: Issued Press Release on O...,False,"[0, 267]","{'hashtags': [], 'symbols': [{'text': 'CRM', '...","<a href=""https://vymcfued1m.execute-api.us-eas...",None,None,...,0,1445072606193983502,1445072606193983502,False,False,en,None,False,True,NaN
1440014230644662272,Mon Sep 20 18:05:21 +0000 2021,1440014230644662272,1440014230644662272,"Bull Forecasts\nSeptember 20, 2021 11:00:00 PD...",False,"[0, 211]","{'hashtags': [], 'symbols': [{'text': 'VLTA', ...","<a href=""https://help.twitter.com/en/using-twi...",None,None,...,0,1440014230644662272,1440014230644662272,False,False,en,None,NaN,NaN,NaN
1431361055494709255,Fri Aug 27 21:00:43 +0000 2021,1431361055494709255,1431361055494709255,$CRM [15s. delayed]: Issued Press Release on A...,False,"[0, 183]","{'hashtags': [], 'symbols': [{'text': 'CRM', '...","<a href=""https://vymcfued1m.execute-api.us-eas...",None,None,...,0,1431361055494709255,1431361055494709255,False,False,en,None,False,True,NaN
1430306935425978368,Tue Aug 24 23:12:01 +0000 2021,1430306935425978368,1430306935425978368,"TICKER: $CRM\nSIZE: 421,996\nPRICE: $259.66\nT...",False,"[0, 77]","{'hashtags': [], 'symbols': [{'text': 'CRM', '...","<a href=""https://www.integromat.com"" rel=""nofo...",None,None,...,0,1430306935425978368,1430306935425978368,False,False,en,None,NaN,NaN,NaN


""


""


In [5]:
crm_df.sort_index(inplace=True)

In [6]:
'''def getTodayTweets(current_df_path)
    
    todaysTweets_df = pd.DataFrame()
    
    current_df_path = Path('./twitter_flair.csv'
    currentTweets_df = pd.read_csv(current_df_path, parse_dates = True, infer_datetime_format=True, index_col='Date')
    hashtag = input('Please input desired hashtag to confirm')
    
    end_date = datetime.datetime.today()
    start_date = today - pd.Timedelta(days=1)
    
    
    #Iterating through the date_range with an API call for each day to maximize data returned 
    # @TODO -- use function to limit API calls to not exceed limit
    
    start_date = pd.Timestamp.isoformat(start_date)
    end_date = pd.Timestamp.isoformat(end_date)
    
    
    # Making the API call to retrieve tweets 

    source = "https://twitter32.p.rapidapi.com/getSearch"

    # These parameters need to be set before running API call... could in the future receive inputs from user
    ## TODO -- if input is received, datetime will need to be formatted correctly for API call

    hashtag = hashtag
    querystring = {"hashtag": hashtag, "start_date": start_date,"end_date": end_date,"lang":"en"}
    headers = {
        'x-rapidapi-host': "twitter32.p.rapidapi.com",
        'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
        }
   
        
    response = requests.get(source, headers=headers, params=querystring).json()
    todaysTweets_df = pd.DataFrame(response['data']['tweets']).T
    updatedTweets_df = pd.concat([todays_Tweets_df, currentTweets])
    display(start_date, end_date)
                           
    
    return updatedTweets_df

updatedTweets_df = getTwitterData('./twitter_flair.csv')
display(tweets_df)'''

'def getTodayTweets(current_df_path)\n    \n    todaysTweets_df = pd.DataFrame()\n    \n    current_df_path = Path(\'./twitter_flair.csv\'\n    currentTweets_df = pd.read_csv(current_df_path, parse_dates = True, infer_datetime_format=True, index_col=\'Date\')\n    hashtag = input(\'Please input desired hashtag to confirm\')\n    \n    end_date = datetime.datetime.today()\n    start_date = today - pd.Timedelta(days=1)\n    \n    \n    #Iterating through the date_range with an API call for each day to maximize data returned \n    # @TODO -- use function to limit API calls to not exceed limit\n    \n    start_date = pd.Timestamp.isoformat(start_date)\n    end_date = pd.Timestamp.isoformat(end_date)\n    \n    \n    # Making the API call to retrieve tweets \n\n    source = "https://twitter32.p.rapidapi.com/getSearch"\n\n    # These parameters need to be set before running API call... could in the future receive inputs from user\n    ## TODO -- if input is received, datetime will need to be

In [7]:
# Parsing and formatting the dataframe to index by date created and isolating the 'full_text' and 'retweet_count' columns for sentiment analysis
df_filtered = crm_df.loc[:,["created_at","full_text", 'retweet_count']]
df_filtered.index = pd.to_datetime(df_filtered['created_at'], utc=True)
df_filtered.index.name = 'Date'
df_filtered.drop(columns='created_at', inplace=True)
df_filtered.sort_index(inplace=True)
df_filtered


,full_text,retweet_count
Date,,
2021-01-21 12:06:39+00:00,"I see tickers +300%, +500% and people still lo...",0
2021-03-30 09:01:55+00:00,$CRM [15s. delayed]: Issued Press Release on M...,0
2021-03-30 23:10:54+00:00,TICKER: $CRM\nSIZE: 442763\nPRICE: $210.0\nTIM...,0
2021-04-14 02:00:17+00:00,TICKER: $CRM\nSIZE: 731863\nPRICE: $228.76\nTI...,0
2021-04-21 12:10:36+00:00,$CRM [15s. delayed]: Issued Press Release on A...,0
2021-04-26 12:06:50+00:00,$CRM [15s. delayed]: Issued Press Release on A...,0
2021-05-25 11:03:42+00:00,$CRM [15s. delayed]: Issued Press Release on M...,0
2021-05-27 22:08:41+00:00,$CRM [15s. delayed]: Issued Press Release on M...,0
2021-06-01 13:15:26+00:00,$CRM [15s. delayed]: Issued Press Release on J...,0


In [8]:
# We need to clean up our tweets so they can be analyzed in string format
# Applying our imported 'clean_text' function to text column of our dataframe
df_filtered["full_text"] = df_filtered["full_text"].apply(clean_text)
df_filtered = df_filtered.sort_index()
df_filtered

,full_text,retweet_count
Date,,
2021-01-21 12:06:39+00:00,I see tickers 300 500 and people still loading...,0
2021-03-30 09:01:55+00:00,CRM 15s delayed Issued Press Release on March ...,0
2021-03-30 23:10:54+00:00,TICKER CRM SIZE 442763 PRICE 210 0 TIME 2021 0...,0
2021-04-14 02:00:17+00:00,TICKER CRM SIZE 731863 PRICE 228 76 TIME 19 00...,0
2021-04-21 12:10:36+00:00,CRM 15s delayed Issued Press Release on April ...,0
2021-04-26 12:06:50+00:00,CRM 15s delayed Issued Press Release on April ...,0
2021-05-25 11:03:42+00:00,CRM 15s delayed Issued Press Release on May 25...,0
2021-05-27 22:08:41+00:00,CRM 15s delayed Issued Press Release on May 27...,0
2021-06-01 13:15:26+00:00,CRM 15s delayed Issued Press Release on June 0...,0


In [9]:


# Get sentiment score for each review
df_filtered['scores_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[0])
# Predict sentiment label for each review
df_filtered['pred_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[1])
# Check the distribution of the score
df_filtered['scores_flair'].describe()
# Change the label of flair prediction to 0 if negative and 1 if positive
#mapping = {'NEGATIVE': 0, 'POSITIVE': 1}
#df_filtered['pred_flair_numerical'] = df_filtered['pred_flair'].map(mapping)
df_filtered['pred_flair'].describe()


count           35
unique           2
top       NEGATIVE
freq            26
Name: pred_flair, dtype: object

In [10]:
#Converting the categorical variable columns into numerical with get_dummies
df_dummies = pd.get_dummies(df_filtered['pred_flair'])
# Dropping the row if the scores_flair confidence level is below 0.75
df_filtered.drop(df_filtered[df_filtered.scores_flair < 0.75].index, inplace=True)
# Creating the 'df_signals' dataframe that we will use to append to our primary dataset, using only the columns with relevant numerical values, resampled by day
df_signals = df_filtered.drop(columns=['scores_flair', 'pred_flair'])
final_df = pd.concat([df_signals, df_dummies]).resample('D').sum()
display(final_df)  
final_df.to_csv('crm_flair.csv')

    
    


,NEGATIVE,POSITIVE
Date,,
2021-01-21 00:00:00+00:00,1.0,0.0
2021-01-22 00:00:00+00:00,0.0,0.0
2021-01-23 00:00:00+00:00,0.0,0.0
2021-01-24 00:00:00+00:00,0.0,0.0
2021-01-25 00:00:00+00:00,0.0,0.0
...,...,...
2022-01-02 00:00:00+00:00,0.0,0.0
2022-01-03 00:00:00+00:00,1.0,0.0
2022-01-04 00:00:00+00:00,0.0,0.0


### After pulling tweet data, performing sentiment analysis using flair and then filtered and resampling the dataframe, we return a dataframe with datetime indices with two columns -- flair_negative, which is the sum of all negative tweets for the day, and flair_positive, which is the sum of all positive tweets for the day -- at some point we could think about factoring in number of retweets or something along those lines to weigh the tweets by their impact, but the results from the API call already filter by this metric, so could be superfluous.